# Interactive Brokers OAuth Test

**Prerequisites:** IBKR OAuth credentials (`ib_oauth`) must be configured in NoETL credential table.

In [ ]:
import requests
import json
import time
from datetime import datetime
import os

# Configuration
NOETL_SERVER = os.getenv('NOETL_SERVER', 'http://localhost:8082')
PLAYBOOK_PATH = 'tests/fixtures/playbooks/oauth/interactive_brokers'
PLAYBOOK_FILE = 'tests/fixtures/playbooks/oauth/interactive_brokers/ib_oauth.yaml'

print(f"NoETL Server: {NOETL_SERVER}")
print(f"Playbook Path: {PLAYBOOK_PATH}")
print(f"Test Time: {datetime.now()}")

## Register Playbook

In [ ]:
# Read playbook content
with open(PLAYBOOK_FILE, 'r') as f:
    playbook_content = f.read()

# Register playbook
response = requests.post(
    f"{NOETL_SERVER}/api/catalog/register",
    json={
        'path': PLAYBOOK_PATH,
        'content': playbook_content
    }
)

result = response.json()
print(f"Registration Status: {result.get('status')}")
print(f"Version: {result.get('version')}")

assert result.get('status') == 'success', f"Registration failed: {result}"

## Execute Playbook

In [ ]:
# Execute playbook
response = requests.post(
    f"{NOETL_SERVER}/api/run/playbook",
    json={'path': PLAYBOOK_PATH}
)

result = response.json()
execution_id = result.get('execution_id')

print(f"Execution ID: {execution_id}")
assert execution_id is not None, f"Failed to start execution: {result}"

# Wait for execution
print("Waiting for execution to complete...")
time.sleep(20)

## Check Execution Events

In [ ]:
# Query events
query = f"""
SELECT node_name, event_type, created_at
FROM noetl.event
WHERE execution_id = {execution_id}
ORDER BY event_id
"""

response = requests.post(
    f"{NOETL_SERVER}/api/postgres/execute",
    json={'query': query, 'schema': 'noetl'}
)

events = response.json().get('result', [])
print(f"\nTotal Events: {len(events)}")
print("\nExecution Flow:")
for event in events:
    node_name, event_type, created_at = event
    print(f"  {node_name}: {event_type}")

## Validate Results

In [ ]:
# Check list_accounts API result
query = f"""
SELECT result->'status', result->'data'->'status_code'
FROM noetl.event
WHERE execution_id = {execution_id}
  AND node_name = 'list_accounts'
  AND event_type = 'action_completed'
"""

response = requests.post(
    f"{NOETL_SERVER}/api/postgres/execute",
    json={'query': query, 'schema': 'noetl'}
)

accounts_result = response.json().get('result', [])
accounts_status = accounts_result[0][1] if accounts_result else None

# Check get_account_summary API result
query = f"""
SELECT result->'status', result->'data'->'status_code'
FROM noetl.event
WHERE execution_id = {execution_id}
  AND node_name = 'get_account_summary'
  AND event_type = 'action_completed'
"""

response = requests.post(
    f"{NOETL_SERVER}/api/postgres/execute",
    json={'query': query, 'schema': 'noetl'}
)

summary_result = response.json().get('result', [])
summary_status = summary_result[0][1] if summary_result else None

print(f"List Accounts Status: {accounts_status}")
print(f"Get Account Summary Status: {summary_status}")

if accounts_status == 200 and summary_status == 200:
    print("\n✅ Test PASSED - IBKR OAuth working")
else:
    print(f"\n⚠️ Test completed with HTTP codes: {accounts_status}, {summary_status}")